# Human detection - Models Comparison

**Warning - Notebook runned on colab.research platform**

Also, don't forget to activate GPU

# Prerequies - download & install

### 1.My **human-detection** package

In [1]:
!git clone https://github.com/artzet-s/human-detection
%cd human-detection
%run setup.py install
# WARNING - You must restart environement after
%cd ..

Cloning into 'human-detection'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 92 (delta 44), reused 76 (delta 28), pack-reused 0
Unpacking objects: 100% (92/92), done.
/content/human-detection
running install
running bdist_egg
running egg_info
creating human_detection.egg-info
writing human_detection.egg-info/PKG-INFO
writing dependency_links to human_detection.egg-info/dependency_links.txt
writing top-level names to human_detection.egg-info/top_level.txt
writing manifest file 'human_detection.egg-info/SOURCES.txt'
writing manifest file 'human_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/human_detection
copying human_detection/json_tool.py -> build/lib/human_detection
copying human_detection/video_frames.py -> build/lib/human_detection
copying human_det

zip_safe flag not set; analyzing archive contents...
human_detection.__pycache__.__init__.cpython-36: module references __path__


creating dist
creating 'dist/human_detection-0.0.1-py3.6.egg' and adding 'build/bdist.linux-x86_64/egg' to it
removing 'build/bdist.linux-x86_64/egg' (and everything under it)
Processing human_detection-0.0.1-py3.6.egg
creating /usr/local/lib/python3.6/dist-packages/human_detection-0.0.1-py3.6.egg
Extracting human_detection-0.0.1-py3.6.egg to /usr/local/lib/python3.6/dist-packages
Adding human-detection 0.0.1 to easy-install.pth file

Installed /usr/local/lib/python3.6/dist-packages/human_detection-0.0.1-py3.6.egg
Processing dependencies for human-detection==0.0.1
Finished processing dependencies for human-detection==0.0.1
/content


# **Restart your environment ! In order to have human_detection package** (CTRL + M)

### 2 **Ultralytics yolov5** github

No need to restart here like we work in the yolov5 directory

In [1]:
import torch
from IPython.display import clear_output

!git clone https://github.com/ultralytics/yolov5  # clone repo

%cd yolov5

%pip install -qr requirements.txt  # install dependencies

clear_output()
print('Setup complete. Using torch %s %s' % (
    torch.__version__, 
    torch.cuda.get_device_properties(0) 
    if torch.cuda.is_available() else 'CPU'))

# To fix PyYAML version error use : 
# pip install --ignore-installed PyYAML

Setup complete. Using torch 1.7.0+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15079MB, multi_processor_count=40)


# Prepare the data

## 1.1 Download youtube video from url with youtube-dl

In [2]:
video_url = "https://youtu.be/h4s0llOpKrU"
video_filename = "../input_video.mp4"

!pip install --upgrade youtube-dl
!youtube-dl -F $video_url
!youtube-dl -f 137 $video_url  -o $video_filename 

     |████████████████████████████████| 1.9MB 14.2MB/s 
[youtube] h4s0llOpKrU: Downloading webpage
[info] Available formats for h4s0llOpKrU:
format code  extension  resolution note
249          webm       audio only tiny   53k , opus @ 50k (48000Hz), 282.96KiB
250          webm       audio only tiny   71k , opus @ 70k (48000Hz), 374.94KiB
140          m4a        audio only tiny  130k , m4a_dash container, mp4a.40.2@128k (44100Hz), 713.10KiB
251          webm       audio only tiny  140k , opus @160k (48000Hz), 738.09KiB
394          mp4        256x144    144p   71k , av01.0.00M.08, 25fps, video only, 363.07KiB
278          webm       256x144    144p   92k , webm container, vp9, 25fps, video only, 497.53KiB
160          mp4        256x144    144p  110k , avc1.4d400c, 25fps, video only, 429.41KiB
395          mp4        426x240    240p  150k , av01.0.00M.08, 25fps, video only, 730.89KiB
242          webm       426x240    240p  224k , vp9, 25fps, video only, 972.57KiB
133          mp4     

## 1.2 Transform video to image frames


In [3]:
import human_detection
from human_detection.video_frames import video_to_frames, \
                                         write_txt_list_of_filename

frames_dir = "../input_frames"
frames_txt = "../input_frames.txt"

video_to_frames(video_filename, frames_dir, False)

## 1.3 Load refs and make a video

In [4]:
from human_detection.video_frames import frames_to_video
from human_detection.draw_box import draw_person_box_on_frames

%cd "/content/"
json_refs = "/content/human-detection/predictions/ref.json"
pred_frames_dir = '/content/ref_frames'
pred_video = '/content/ref_video.mp4'

print("Draw person box on frames")
draw_person_box_on_frames(json_refs, 
                          pred_frames_dir,
                          confidence=0.50)

print("Convert frames to video")
frames_to_video(pred_frames_dir, 
                video_filename=pred_video, 
                fps=25)

/content
Draw person box on frames
Convert frames to video


# Models Comparison

## Utilities

In [5]:
%cd "/content/yolov5"

import os
import os.path
import glob
import cv2
import json
import numpy
import torch
import collections
import pandas

from IPython.display import Image, display

from utils.general import box_iou, xywh2xyxy
from utils.metrics import ap_per_class

def load_human_bbox(objects):
  bbox = list()
  for obj in objects:
    if obj["name"] == "person":
      bbox.append(list(obj["relative_coordinates"].values()) + [obj["confidence"]])
  return numpy.array(bbox)
      

def compare_result(json_preds, 
                   json_refs, 
                   confidence=0.50,
                   img_size=(1080, 1920)):

  with open(json_preds) as f:
    preds = json.load(f)

  with open(json_refs) as f:
    targets = json.load(f)
  
  preds.sort(key= lambda e: int(os.path.basename(e['filename'][:-4])))
  targets.sort(key= lambda e: int(os.path.basename(e['filename'][:-4])))

  stats = list()
  for pred, target in zip(preds, targets):
    
    filename = None
    if os.path.basename(pred["filename"]) != os.path.basename(target["filename"]):
      print("ERROR between {} and {}".format(pred, target))
      return -1
    else:
      img_height, img_width = img_size

      # human only
      hpred = load_human_bbox(pred["objects"])
      htarget = load_human_bbox(target["objects"])

      if htarget.size == 0: # don't care
          continue

      if hpred.size == 0:
        if htarget.size > 0:
          #print("Not detected in : ", target['filename'])
          correct = numpy.zeros((htarget.shape[0], 1), dtype=bool)
          #correct = [False] * htarget.shape[0]
          zeros = [0] * htarget.shape[0]
          stats.append((correct, zeros, zeros, zeros))
          #stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
      else:

        htbox = xywh2xyxy(htarget[:, :4])
        htbox[:, 0] *= img_width
        htbox[:, 1] *= img_height
        htbox[:, 2] *= img_width
        htbox[:, 3] *= img_height

        hpbox = xywh2xyxy(hpred[:, :4])
        hpbox[:, 0] *= img_width
        hpbox[:, 1] *= img_height
        hpbox[:, 2] *= img_width
        hpbox[:, 3] *= img_height
        
        hpbox = torch.from_numpy(hpbox)
        htbox = torch.from_numpy(htbox)
        ious, tindex = box_iou(hpbox[:, :4], htbox[:, :4]).max(1)

        # Append detections
        detected_set = set()
        correct = numpy.zeros((len(tindex), 1), dtype=bool)
        for j in (ious > confidence).nonzero(as_tuple=False):
          d = tindex[j]  # detected target
          if d.item() not in detected_set:
            detected_set.add(d.item())
            correct[j, 0] = ious[j] > confidence 

        tcls = [0] * len(tindex)
        stats.append((correct, hpred[:, 4], tcls, tcls))


  stats = [numpy.concatenate(x, 0) for x in zip(*stats)]  # to numpy
  if len(stats) and stats[0].any(): #a least one True
    p, r, ap, f1, ap_class = ap_per_class(*stats)
    return [p[0][0], r[0][0], ap[0][0], f1[0][0]]
  else:
    return [0, 0, 0, 0]

/content/yolov5


## Run comparison

In [17]:
confidence = 0.25

data = list()
for json_preds in glob.glob("../human-detection/predictions/yolo*.json"):  
  name = os.path.basename(json_preds)
  r = compare_result(json_preds=json_preds,
                     json_refs=json_refs,
                     confidence=confidence)
  data.append([name, confidence] + r)

## Display Results

In [18]:
df = pandas.DataFrame(data, columns=['name', 'confidence', 'precision', 'recall', 'mAP', 'F1'])
df = df.round(decimals=2)
df.head(10)

,name,confidence,precision,recall,mAP,F1
0,yolov5x_result.json,0.25,0.93,0.84,0.84,0.88
1,yolov5l_result.json,0.25,0.94,0.85,0.84,0.89
2,yolov5s_result.json,0.25,0.92,0.78,0.75,0.84
3,yolov4_result.json,0.25,0.94,0.87,0.87,0.90
4,yolov3_result.json,0.25,0.90,0.80,0.79,0.84
5,yolov4-csp_result.json,0.25,0.94,0.86,0.86,0.90
6,yolov4x-mish_result.json,0.25,0.95,0.87,0.87,0.91
7,yolov5m_result.json,0.25,0.93,0.85,0.84,0.89
8,yolov4-tiny_result.json,0.25,0.90,0.69,0.69,0.79
9,result.json,0.25,0.75,0.73,0.72,0.74


### The best model is :

In [ ]:
print(df.loc[df['mAP'].idxmax()])

name          yolov4_result.json
confidence                  0.25
precision                   0.94
recall                      0.87
mAP                         0.87
F1                           0.9
Name: 3, dtype: object
